In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd 

In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups



newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

vectorizer = CountVectorizer(stop_words='english',max_features=2000)
trainX = vectorizer.fit_transform(newsgroups_train.data)
testX = vectorizer.transform(newsgroups_test.data)
voc = vectorizer.vocabulary_
print (trainX.shape)
testX.shape

(11314, 2000)


(7532, 2000)

In [2]:
temp = 0.1

X = tf.placeholder(tf.float32, shape=[None,2000])
x = tf.placeholder(tf.float32, shape=[None,2000])

batch = tf.placeholder(tf.int32, shape=())

#WH = tf.Variable(tf.zeros([120,90]),name='WH')
#bH = tf.Variable(tf.zeros([90]),name='biasH')

def decoder(x,name="classDecoder"):

    #with tf.variable_scope(name):

        W1 = tf.Variable(tf.random.uniform([2000,300],-0.5,0.5),name='W1',dtype=tf.float32)
        b1 = tf.Variable(tf.zeros([300]),name='bias1',dtype=tf.float32)

        layer1 = tf.nn.sigmoid(tf.matmul(x,W1) + b1)

        W2 = tf.Variable(tf.random.uniform([300,250],-0.5,0.5),name='W2')
        b2 = tf.Variable(tf.zeros([250]),name='bias2',dtype=tf.float32)

        layer2 = tf.nn.sigmoid(tf.matmul(layer1,W2) + b2)

        W3 = tf.Variable(tf.random.uniform([250,50],-0.05,0.05),name='WMu')
        b3 = tf.Variable(tf.zeros([50]),name='biasMu',dtype=tf.float32)

        W4 = tf.Variable(tf.random.uniform([250,50],0,0),name='WSigma')
        b4 = tf.Variable(tf.zeros([50]),name='biasSigma',dtype=tf.float32)

        mu = tf.matmul(layer2,W3) + b3
        log_sigma = tf.matmul(layer2,W4) + b4

        return mu,log_sigma

def decoder2(hs,name="C"):

    with tf.variable_scope(name):
    
        W1 = tf.Variable(tf.random.normal([2000,300],-0.05,0.05),name='W1')
        b1 = tf.Variable(tf.zeros([300]),name='bias1',dtype=tf.float32)

        
        layer1 = tf.nn.sigmoid(tf.matmul(hs,W1) + b1)

        W2 = tf.Variable(tf.random.uniform([300,150],-0.05,0.05),name='W2')
        b2 = tf.Variable(tf.zeros([150]),name='bias2',dtype=tf.float32)

        layer2 = tf.nn.relu(tf.matmul(layer1,W2) + b2)

        W5 = tf.Variable(tf.random.uniform([150,4],-0.00,0.00),name='WC')
        b5 = tf.Variable(tf.zeros([4]),name='biasC',dtype=tf.float32)
        

        C = tf.nn.softmax(tf.sigmoid(tf.matmul(layer2,W5) + b5))

        return C


def encoder(x,h,name='ClassEncoder'):

     with tf.variable_scope(name):

          R = tf.Variable(tf.random.uniform([50,2000],-0.01,0.01),name='RWord')
          b = tf.Variable(tf.zeros([2000]),name='biasWord')

          probs = tf.log(tf.nn.softmax(tf.matmul(h,R)+b))

          final = tf.reduce_sum(tf.multiply(probs,x),axis=1)

          return -final,R


networks = []
networks2 = []
ss = []
df = []
pi = tf.distributions.Categorical(probs=[0.25,0.25,0.25,0.25])
mu,log_sigma = decoder(x)
eps = tf.random_normal((batch,50), 0,1)
h = mu+tf.multiply(tf.exp(log_sigma),eps)
rtt = []

for l in range(0,4):


    probsE,goes = encoder(x,h,name="Class_"+str(l))
    networks.append(probsE)
    df.append(h)
    rtt.append(goes)
    #networks2.append(-0.5 * tf.reduce_sum(1 - tf.square(mu) + 2 * log_sigma - tf.exp(2 * log_sigma), 1))


C = decoder2(x)
eps2 = tf.random_uniform((batch,4), 0,1)
eps2 = -tf.log(-tf.log(eps2))

C2 = tf.log(C)+eps2
C2 = tf.nn.softmax(C2/temp,axis=1)

enc2 = networks*tf.transpose(C2)
enc = tf.reduce_sum(enc2,axis=0)


kld = -tf.reduce_sum(pi.probs*tf.log(pi.probs/C),axis=1)
kld2 = -0.5 * tf.reduce_sum(1 - tf.square(mu) + 2 * log_sigma - tf.exp(2 * log_sigma), 1) 
train = enc
train1 = train + kld + kld2
train2 = tf.reduce_mean(train1)

optimizer = tf.train.AdamOptimizer(learning_rate=0.0013)

step = optimizer.minimize(train2)
saver = tf.train.Saver()
sess = tf.Session()
print('Initializing...')

init = tf.global_variables_initializer()
sess.run(init)
#saver.save(sess,path+'first_save')
#feed_dict = {x:trainX[0].reshape(1,2000)}
#sess.run(step,feed_dict=feed_dict)
#sess.run(train,feed_dict=feed_dict)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Initializing...


In [4]:
from random import shuffle
indices = np.arange(trainX.shape[0]) #gets the number of rows 
shuffle(indices)
print (indices)

[ 2273 10444  1566 ...  4967   926  2073]


In [6]:
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cosine

for ll in range(0,25):

    #print (begin,end,"<===>")
    begin = 0
    end = 32
    cont = 0
    while end <=11314:

        select = indices[begin:end]
        feed_dict = {x:trainX[select].toarray().reshape(32,2000),batch:32}
        sess.run(step,feed_dict=feed_dict)

        begin = end
        end += 32
            
        cont +=1

    print ("########")
    feed_dict = {x:testX.toarray().reshape(7532,2000),batch:7532}
    a = sess.run(train1,feed_dict=feed_dict)
    ppr  = np.exp(np.sum(a/np.sum(testX.toarray(),axis=1))/7532.0)
    print (ppr)
    #1291

########
1180.9269970905577
########
1101.2138906030611
########
1060.1107094710405
########
1020.7050474951199
########
997.660721193846
########
973.1300430439517
########
947.8046660994715
########
929.3844054233617
########
911.8034418327793
########
893.4661076489798
########
881.324936427153
########
867.6051977156646
########
858.6720843067258
########
846.5714556140106
########
834.90579640781
########
820.9018981056438
########
815.8885983913359
########
808.2972204705154
########
799.2108790308259
########
793.1731944434033
########
782.6005098094555
########
774.2380411109897
########
772.3034652540958
########
760.0584932134313
########
755.4571657107845
